In [ ]:
#Most functional build
#libraries
import numpy as np
import matplotlib.pyplot as plt
#Fret Association Block
 #[neck stock length,scale length,max frets over estimate]:neck_fret_info input format
    #[fret loc data relative to nut=0, list of fret numbers, number of frets able to fit in stock length]
def scale_len_fret_locs(neck_fret_info):#list inputs of numbers
    fret_loc_data=[0]
    #[neck stock length,scale length,max frets over estimate]:neck_fret_info input format
    #[fret loc data relative to nut=0, list of fret numbers, number of frets able to fit in stock length]
    neck_stock_len,scale_len,max_frets_over=[neck_fret_info[0],neck_fret_info[1],neck_fret_info[2]]
    body_string_span=scale_len-neck_stock_len
    if scale_len==0:
        
            print("scale length bruh")
    else:
        for n in range(1,max_frets_over):
            prev=fret_loc_data[n-1]#based on recursive formula of prev fret and scale len
            fret_loc_data.append(((scale_len-prev)/17.817)+prev)
        if fret_loc_data[-1]>neck_stock_len:#checks if last fret is out of stock length
            for i in range(len(fret_loc_data)):#if so for every data point in fret loc data original
                if fret_loc_data[i]>neck_stock_len:#if that data point is out of stock length
                    fret_loc_data=fret_loc_data[:i]#cut the list off there
                    break

        true_max_frets=len(fret_loc_data)
        fret_number_data=list(range(true_max_frets))
    return fret_loc_data, fret_number_data,true_max_frets
#Polygon Characterization Block

#AI suggestion:
def polygon_area_and_inertia(vertices):
    """
    Inputs: generic list of tuples, not arrays
    Calculate area, centroid, and area moments of inertia (Ix, Iy, Ixy)
    for a polygon defined by vertices in order (clockwise or counterclockwise).

    non AI note: Don't enter the first vertex after the very last one; function closes the loop automatically.

    Parameters:
        vertices: List of (x, y) tuples in consistent order.

    Returns:
        area: Signed area (positive if CCW order)
        cx, cy: Centroid coordinates
        Ix, Iy, Ixy: Moments of inertia about the origin
    Ai reasoning: 
    uses shoelace/gauss area algorithm to sum determinates of position vectors centered on global origin, not centroid of polygon
    second func called to correct radially dependant properties for centroidal axes ref frame
    
    """
    if len(vertices) < 3:
        raise ValueError("A polygon must have at least 3 vertices.")

    # Close the polygon loop
    vertices = vertices + [vertices[0]]

    A = 0.0
    Cx = 0.0
    Cy = 0.0
    Ix = 0.0
    Iy = 0.0
    Ixy = 0.0

    for i in range(len(vertices) - 1):
        v0= (vertices[i])  
        v1 = (vertices[i+1])
        [x0, y0, x1, y1 ]= [v0[0], v0[1], v1[0], v1[1]]

        cross = x0 * y1 - x1 * y0
        A += cross
        Cx += (x0 + x1) * cross
        Cy += (y0 + y1) * cross
        Ix += (y0**2 + y0 * y1 + y1**2) * cross
        Iy += (x0**2 + x0 * x1 + x1**2) * cross
        Ixy += (x0 * y1 + 2 * x0 * y0 + 2 * x1 * y1 + x1 * y0) * cross

    A *= 0.5
    if A == 0:
        raise ValueError("Polygon area is zero — check vertex order or coordinates.")

    Cx /= (6 * A)
    Cy /= (6 * A)
    Ix /= 12.0
    Iy /= 12.0
    Ixy /= 24.0

    return abs(A), Cx, Cy, abs(Ix), abs(Iy), abs(Ixy)


def polygon_centroid_properties(vertices):
    """
    Inputs: generic list of tuples, not arrays
    Compute moments of inertia about the centroidal axes using the parallel axis theorem to shift refrence frame from outside origin 
    (probably) the location of neutral axis to make our model more accurate given what you can easily measure and account for 
    Calls prev func to fetch props
    """
    A, Cx, Cy, Ix_origin, Iy_origin, Ixy_origin = polygon_area_and_inertia(vertices)

    # Shift to centroid
    Ix_centroid = Ix_origin - A * (Cy ** 2)
    Iy_centroid = Iy_origin - A * (Cx ** 2)
    Ixy_centroid = Ixy_origin - A * Cx * Cy

    return [ A,
         Cx, Cy,
        Ix_centroid,
         Iy_centroid,
         Ixy_centroid]

#testing - works?



#new chug vertex func based on old one, only returns vertexes but converts coords to non-grid units for later use

def chug_vertex_inputs(true_max_frets, max_grid_estimate,grid_to_unit_conversion_factor):
    """
    Collect vertex coordinates for each fret and return as list of polygon-ready tuples.
    
    User enters x,y pairs as floats separated by commas.
    Example input: "0,0" for point (0,0), "1,1" for point (1,1), etc.
    Enter "done" when finished with current fret.
    
    Args:
        true_max_frets: Number of frets to sample
        max_grid_estimate: Approximate number of vertices expected per fret
    
    Returns:
        List of lists, where each inner list contains (x,y) tuples for that fret
    """
    all_fret_vertices = []
    
    for fret in range(true_max_frets):
        print(f"\n{'='*60}")
        print(f"Fret {fret} - Enter vertex coordinates")
        print(f"Expected ~{max_grid_estimate} vertices (quadrilateral/polygon)")
        print("Format: Enter each coordinate as 'x,y' (e.g., '0,0')")
        print("Type 'done' when finished with this fret")
        print(f"{'='*60}\n")
        
        vertices = []
        vertex_count = 0
        
        while True:
            user_input = input(f"Vertex {vertex_count}: ").strip()
            #didn't know claude could get you emojis lmao
            if user_input.lower() == 'done':
                if len(vertices) < 3:
                    print(f"⚠️  Warning: Only {len(vertices)} vertices entered. Need at least 3 for a polygon.")
                    retry = input("Continue anyway? (y/n): ").strip().lower()
                    if retry != 'y':
                        continue
                break
            
            try:
                # Parse "x,y" format
                parts = user_input.split(',')
                if len(parts) != 2:
                    print("❌ Invalid format. Please enter as 'x,y' (e.g., '1.5,2.3')")
                    continue
                
                x = float(parts[0].strip())
                y = float(parts[1].strip())
                vertices.append((x, y))
                vertex_count += 1
                print(f"✓ Added vertex {vertex_count}: ({x}, {y})")
                
            except ValueError:
                print("❌ Invalid input. Please enter two numbers separated by comma (e.g., '0,0')")
        
        all_fret_vertices.append(vertices)
        print(f"✓ Fret {fret} complete with {len(vertices)} vertices\n")
        all_fret_vertices_units=all_fret_vertices*grid_to_unit_conversion_factor
    return all_fret_vertices,all_fret_vertices_units

#Claude AI suggestion for test data generation
def chug_vertex_inputs_batch(true_max_frets, test_data=None):
    """
    Alternative: Load test vertex data instead of interactive input.
    Useful for debugging without user input.
    
    Args:
        true_max_frets: Number of frets
        test_data: List of vertex lists. If None, uses simple test polygons.
    
    Returns:
        List of vertex tuple lists
    """
    if test_data is None:
        # Generate simple test data: rectangles with slight variations
        test_data = []
        for fret in range(true_max_frets):
            # Create a trapezoid shape that gradually changes (simulating neck taper)
            width = 0.05 - (fret * 0.001)  # Narrower toward body
            height = 0.02
            x_offset = fret * 0.001
            
            vertices = [
                (x_offset, 0.0),           # bottom left
                (x_offset + width, 0.0),   # bottom right
                (x_offset + width*0.9, height),  # top right
                (x_offset + width*0.1, height),  # top left
            ]
            test_data.append(vertices)
    
    return test_data

#Claude AI suggestion for improved data handling: use classes to make accessing/adding info easier
#Class variables: 
 # fret number: fret_n
# fret location (m):fret_loc
# cross-section area (m²):area
# Ix - transverse bending:moi_transverse
# Iy - double normal bending:moi_double_normal
# Ixy - torsional::moi_longitudinal
class FretData:
    """
    Container for fret geometric properties with guaranteed alignment.
    Ensures fret_n, fret_loc, area, and MOI properties always have matching indices.
    """
    #Class variables: 
    # fret number: fret_n
    # fret location (m):fret_loc
    # cross-section area (m²):area
    # Ix - transverse bending:moi_transverse
    # Iy - double normal bending:moi_double_normal
    # Ixy - torsional::moi_longitudinal
    def __init__(self):
        self.fret_n = []          # fret number
        self.fret_loc = []        # fret location (m)
        self.area = []            # cross-section area (m²)
        self.moi_transverse = []  # Ix - transverse bending
        self.moi_double_normal = [] # Iy - double normal bending
        self.moi_longitudinal = [] # Ixy - torsional
    
    def add_fret(self, fret_num, location, a, ix, iy, ixy):
        """Add a complete fret data point - all or nothing"""
        self.fret_n.append(fret_num)
        self.fret_loc.append(location)
        self.area.append(a)
        self.moi_transverse.append(ix)
        self.moi_double_normal.append(iy)
        self.moi_longitudinal.append(ixy)
    
    def __len__(self):
        return len(self.fret_n)
    
    def get_segments(self):
        """Return list of adjacent fret pairs for gradient calculation"""
        if len(self) < 2:
            return []
        
        segments = []
        for i in range(len(self) - 1):
            segment = {
                'fret_pair': (self.fret_n[i], self.fret_n[i+1]),
                'delta_loc': self.fret_loc[i+1] - self.fret_loc[i],
                'delta_area': self.area[i+1] - self.area[i],
                'delta_moi_t': self.moi_transverse[i+1] - self.moi_transverse[i],
                'delta_moi_dn': self.moi_double_normal[i+1] - self.moi_double_normal[i],
                'delta_moi_l': self.moi_longitudinal[i+1] - self.moi_longitudinal[i],
            }
            segments.append(segment)
        return segments

# REFACTORED: works
def geom_grad_div_rel_to_uncert_refactored(tol_length, tol_scale, fret_data):
    """
    Calculate gradient divergence relative to measurement uncertainty.
    
    Args:
        tol_length: Length measurement tolerance (m)
        tol_scale: Grid spacing tolerance (m)
        fret_data: FretData object with aligned properties
    
    Returns:
        list of dicts with gradient analysis for each segment
    """
    # Calculate tolerance bounds
    tol_area = np.sqrt(2) * tol_scale**2  # dA from dx, dy
    tol_moi = np.sqrt((2*tol_area*tol_scale)**2 + (tol_area*tol_scale**2)**2)#moi is x^2 dA
    
    max_grad_area = tol_area / tol_length
    max_grad_moi = tol_moi / tol_length
    
    results = []
    
    # Calculate gradients between each pair of frets
    for segment in fret_data.get_segments():
        delta_loc = segment['delta_loc']
        
        if delta_loc == 0:
            print(f"Warning: Zero distance between frets {segment['fret_pair']}")
            continue
        
        # Calculate actual gradients
        grad_area = segment['delta_area'] / delta_loc
        grad_moi_t = segment['delta_moi_t'] / delta_loc
        grad_moi_dn = segment['delta_moi_dn'] / delta_loc
        grad_moi_l = segment['delta_moi_l'] / delta_loc
        
        # Calculate divergence as percentage
        div_area = (((abs(grad_area) - max_grad_area)*100 / max_grad_area)+100) if max_grad_area != 0 else 0
        div_moi_t = (((abs(grad_moi_t) - max_grad_moi)*100 / max_grad_moi)+100) if max_grad_moi != 0 else 0
        div_moi_dn = (((abs(grad_moi_dn) - max_grad_moi)*100 / max_grad_moi)+100) if max_grad_moi != 0 else 0
        div_moi_l = (((abs(grad_moi_l) - max_grad_moi)*100 / max_grad_moi)+100) if max_grad_moi != 0 else 0
        
        results.append({
            'frets': segment['fret_pair'],
            'location_span': delta_loc,
            'area_gradient': grad_area,
            'area_divergence_%': div_area,
            'moi_transverse_gradient': grad_moi_t,
            'moi_transverse_divergence_%': div_moi_t,
            'moi_double_normal_gradient': grad_moi_dn,
            'moi_double_normal_divergence_%': div_moi_dn,
            'moi_longitudinal_gradient': grad_moi_l,
            'moi_longitudinal_divergence_%': div_moi_l,
        })
    
    return results

# ============================================================================
# WORKFLOW: How to collect fret data and compute geometric properties
# ============================================================================

# Step 1: Define neck parameters
#must be in meters for consistency
#enter in inches and convert if needed
input=[10, 10, 5]
conversion_factor=0.0254#inch to m
neck_fret_info=[input[0]*conversion_factor,input[1]*conversion_factor,input[2]]
# [neck stock length, scale length, max frets over estimate]
fret_locations, fret_numbers, true_max_frets = scale_len_fret_locs(neck_fret_info)
fb_width = 3.5 *conversion_factor  # inches to meters

#got rid of 1/4 factor dividing fb width, wouldn't it just be the fb width divided by square unit len?

print(f"Scale setup: {true_max_frets} frets, locations (m): {[f'{x:.4f}' for x in fret_locations]}\n")

# Step 2: Collect vertex data for each fret
# Option A: Interactive input (uncomment to use)

grid_to_unit_conversion_factor=conversion_factor/5
max_grid_estimate = round(fb_width / grid_to_unit_conversion_factor)#works?
#vertex_list,vertex_list_meters = chug_vertex_inputs(true_max_frets, max_grid_estimate,grid_to_unit_conversion_factor)

# Option B: Use test data (for debugging without user input)
vertex_list_meters = chug_vertex_inputs_batch(true_max_frets)

print(f"\nVertex data collected: {true_max_frets} frets")
for i, verts in enumerate(vertex_list_meters):
    print(f"  Fret {i}: {len(verts)} vertices - {verts}")

# Step 3: Compute polygon properties for each fret
print("\nComputing polygon properties...")
polygon_results = []
for i, vertices in enumerate(vertex_list_meters):
    try:
        fret_props = polygon_centroid_properties(vertices)
        polygon_results.append(fret_props)
        a, cx, cy, ix, iy, ixy = fret_props
        print(f"  Fret {i}: A={a:.6f}, Cx={cx:.4f}, Cy={cy:.4f}, Ix={ix:.8f}, Iy={iy:.8f}")
    except Exception as e:
        print(f"  ❌ Fret {i} failed: {e}")

# Step 4: Populate FretData container
frets = FretData()
for i, (fret_num, fret_loc) in enumerate(zip(fret_numbers, fret_locations)):
    if i < len(polygon_results):
        a, cx, cy, ix, iy, ixy = polygon_results[i]
        frets.add_fret(fret_num, fret_loc, a, ix, iy, ixy)

print(f"\n✓ Loaded {len(frets)} frets into FretData container\n")
tol_len = 2.54/800#1/8 inch in meter

gradient_results = geom_grad_div_rel_to_uncert_refactored(tol_len, grid_to_unit_conversion_factor, frets)

# Step 5: Display results
print("="*80)
print("GRADIENT RESOLUTION ANALYSIS AS PERCENT +- REL TO MAX TOOL PRECISION")
print("="*80)
for result in gradient_results:
    print(f"\nFrets {result['frets'][0]} → {result['frets'][1]}:")
    print(f"  Location span: {result['location_span']:.4f} m")
    print(f"  Area Precision: {result['area_divergence_%']:.1f}%")
    print(f"  MOI (transverse) Precision: {result['moi_transverse_divergence_%']:.1f}%")
    print(f"  MOI (double normal) Precision: {result['moi_double_normal_divergence_%']:.1f}%")
    print(f"  MOI (longitudinal) Precision: {result['moi_longitudinal_divergence_%']:.1f}%")


#debug notes
#claude code: works I guess
    #test with all squares of unit length, lowk bangs that out
# Step 2: Define measurement tolerances and compute gradient divergence
# Assuming a grid resolution of 1/5 inch = 0.000508 m
    #literally just is a grid resolution square resolultion every 5mm
    #units are in meters with no issues?
#outputs a 100% divergence of all properties for this test case...
    #seems to suggest that the gradient changes are much less significant than the measurement uncertainty, making a 100% convergence with known resolution limits?
    #changed output to make more sence 
#Now outputs that the actual gradient is 0.1% greater than the resolution of the tool at 5mm len res, same grid res, previously was 120 
#with 1mm len res area precision is 100% max theoretical gradient res
#after regiggering error calcs, 5mm len res equals 99% max theoretical res 


#vertex batch generation is making trap tapering down of 0.05 m maax height and span, equal to about 10 grid squares? 
#1mm length res is +0.2% rel to max area res 0.0% from max mois...

#still like +0.2% area and 0's for the rest

#after fixing  area error calc (was previously sqrt(2)*tol grid) to actually be of area unit sqrt(2)tol grid ^2
    #with 1 cm length res and default grid res, area gradient is 30-40%  larger than peak precision, moi double norm +1.7%, rest <1% 
        #1mm length res goes to 3.5%
        #1/8 inch length res goes to 11 percent, MOI long starts to be below max res limit
        #mois really don't be changin since data is just slightly differing traps



Scale setup: 5 frets, locations (m): ['0.0000', '0.0143', '0.0277', '0.0404', '0.0524']


Vertex data collected: 5 frets
  Fret 0: 4 vertices - [(0.0, 0.0), (0.05, 0.0), (0.045000000000000005, 0.02), (0.005000000000000001, 0.02)]
  Fret 1: 4 vertices - [(0.001, 0.0), (0.05, 0.0), (0.0451, 0.02), (0.005900000000000001, 0.02)]
  Fret 2: 4 vertices - [(0.002, 0.0), (0.05, 0.0), (0.045200000000000004, 0.02), (0.0068000000000000005, 0.02)]
  Fret 3: 4 vertices - [(0.003, 0.0), (0.05, 0.0), (0.04530000000000001, 0.02), (0.0077, 0.02)]
  Fret 4: 4 vertices - [(0.004, 0.0), (0.05, 0.0), (0.045399999999999996, 0.02), (0.0086, 0.02)]

Computing polygon properties...
  Fret 0: A=0.000900, Cx=0.0250, Cy=0.0096, Ix=0.00000003, Iy=0.00000015
  Fret 1: A=0.000882, Cx=0.0255, Cy=0.0096, Ix=0.00000003, Iy=0.00000014
  Fret 2: A=0.000864, Cx=0.0260, Cy=0.0096, Ix=0.00000003, Iy=0.00000014
  Fret 3: A=0.000846, Cx=0.0265, Cy=0.0096, Ix=0.00000003, Iy=0.00000013
  Fret 4: A=0.000828, Cx=0.0270, Cy=0.0096,

In [3]:
#Big picture
#Function to accept discretized neck geometry, calculate geometric parameters per cross section
    #treats neck top surf as height datum, all polygons share upper vertex Y coords
    #nodes ordered clockwise, looking down at body from head starting from top left surf of neck top surf

    #A. Fret Profile Assembly Block:
        #Will use grid system with uniform square sizes, rounded to nearest grid coord initially
                #this case engineering paper: 0.2 inch sides
        #1. Asks for user input of neck stock, scale len, overestimate of desired number of frets
            
        #2. User input of current fret being sampled
        #Per fret selected:
        #3. Store polygon vertex inputs in local grid coords
        #4. Convert local grid coords into actual unit (in or m) 
        #5. Compute geometric props
        #6. Append list of fret number, fret loc relative to nut, geometric props
            #A. Rows of fret associated data stacked in assembly
                #no idea what function is needed lol
            #B. Frets lacking inputs indicated by some condition: None
            #C. updates based on changes of inputs
                
                #sublocks needed?:
                    #A. menu window GUI
                    #B. input entering 
                    #C. button functionality
                    #D. input retrieval/processing
                    #E. Labeling stuff in window
        #7. Export/return assembled list []

    #B. Fret Association Block
        # Based on fret positions
        #1: Fret loc/numbering association based on scale length and neck length
            #Same logic for fretless option
        
    #C. Polygon Characterization Block
        #Input:
        #Vertexes of neck profiles
        #Scaling factor for grid coord to actual length units
        #Fret number associated with profile
        #Output:
            #Area 
            #MOIs
            #Centroids 

    


In [ ]:
#Fret Profile Assembly Block:
#measurment inputs to data: 
# - Scale len fret locs=[fret loc data, fret num data, true max fret]
    #fret in question: F
# - Polygon centroid properties on F=[A, Cx, Cy,Ix_centroid,Iy_centroid,Ixy_centroid]    
    #Input: vertexes, grid scale factor
    

#beam el inputs:
#beam_stiffness_matrix(L, E, I,A,N_DOF_G_PER_ELEMENT)
    #assumes both nodes have same area/moi props...C1= AE/L and C2=EI/L^3 trans/rot stiffness conts
    # introduces uncert when span of element becomes larger, larger divergence from nom/averaged val
    # if props are func of length position; C1=A(L)/L =f(L)
    # (A_gradient*L)+A_nom=true_A
        #if A_gradient small, just use A_nom
        #if not, split initial elements into 2 elements, each with A_nom=A_true at nearest node?
            #inputs: detla A tol, delta I tol, delta L tol




In [ ]:
#scrapbin
#old polygon prop and vertex func
def chug_polygon_props(true_max_frets,max_grid_estimate):
    print(f"Max Grid Points {max_grid_estimate}:\n")
    all_fret_params=[]
    for fret in range(true_max_frets):
        print(f"Enter coord list for fret {fret} as list of [x,y] formatted tuples:\n")
        vertex_list=[]
        for grid_point in range(max_grid_estimate*2):
          
            vertex=float(input("Enter as coord: "))#x,y input as floats?
            vertex_list.append(vertex)

            print(f"Vertex List Entered for fret {fret}:{vertex_list}\n")
    #must convert vertex list to list of tuples for func input
        vertex_list=[(vertex_list[i],vertex_list[i+1]) for i in range(len(vertex_list)-1)]
        single_fret_params=polygon_centroid_properties(vertex_list)#6 item list [0-6]
        all_fret_params.append(single_fret_params)#append to master list of 6*true_max_frets items[0-true_max_frets][0-6]
        #[A, Cx, Cy, Ix, Iy, Ixy]
    return all_fret_params


#old grad divergence command
def mesh_advice(mesh_refinement_params,props=None):
    [tol_len,tol_grid,err_spec,true_max_frets]=mesh_refinement_params
    for n in range(1,true_max_frets):
        
  
        #must check if props if test data not active
        if props is not None:
          pass
        else:
            print('No test or actual props entered or active')
            break
        grads=geom_grad_div_rel_to_uncert(tol_len,tol_grid,props)
        #don't ask me why this indexing looks like that 
        #f'{float(f"{num:.ng}"):g} float with n sig figs no trailing zeros
        print(f"Fret {n}: Gradient Divergence From Resolution Limit %\nArea:{float(f'{grads[n-1][1][0][0]:.3g}'):g}% \nMOI [transverse bending, double normal bending, torsional]:{float(f'{grads[n-1][1][0][1]:.3g}'):g}%")

#old gradient uncert func
def geom_grad_div_rel_to_uncert(tol_length,tol_scale,props):
#inputs: props over length: A, I , L of each element
#length measurement tolerance determines other param sensitivities
#assuming uniform grid sizing for arera/scale res
    #assumes orthogonal contributions of error parameters (dx,dy) to dA,dmoi
        #sum of squares square rooted, if all are the equal the base 
    tol_area=np.sqrt(2)*tol_scale**2#dA=dx*dy
    tol_moi=np.sqrt((2*tol_area*tol_scale)**2+(tol_area*tol_scale**2)**2)#dmoi=(dx)^2*dA
    #getting rid of all array formatting to see if that was the issue
        #lists cant be operated on like this
            #back to this issue after getting rid of all array formatting and changing list indexing
            #unsporroted operand type(s) / specifically? for a list and float combo specifically?
                #got rid of brackets of num, not list of list anymore?
                    #unsupported operators for tuple float combo now 
    max_gradients=[tol_area/(tol_length),tol_moi/(tol_length)]
    #x ordered list of geometry vals with corresponding L
        #eff length could be default max to min fret for one element over whole neck
        #or equal to fret spacing for one per fret
        
    [fret_n,fret_loc,area_eff,moi_eff]=[props[0],props[1],props[2],props[3]]

    #commented out to see if array formatting is the issue
    #[fret_n,fret_loc,area_eff,moi_eff]=np.array(fret_n),np.array(fret_loc),np.array(area_eff),np.array(moi_eff)
    gradient_divergence=[]
    for n in range(len(fret_n)):
        #hoepfully breaks before overindexing
        if n==len(fret_n)-1:
            break
        else:
            #after changing max gradients list formatting this part now overindexes its list indices with the n+1
                #still overindexes even with the conditional?
            area_gradient=(area_eff[n+1]-area_eff[n])/(fret_loc[n+1]-fret_loc[n])
            moi_gradient=(moi_eff[n+1]-moi_eff[n])/(fret_loc[n+1]-fret_loc[n])
            #doesn't like if you make this an array
            actual_grads=([area_gradient,moi_gradient])
            #doesn't like elements being set with sequence to numpy array
            #isn't it a list tho?
            #Doesn't like this part specifically...
                #still doesn't like it even if you don't make the variable an array, just putting arrays being operated on does it apparently?
            proportional_gradient_change=([(max_gradients-actual_grads)/max_gradients])
            gradient_divergence.extend(proportional_gradient_change)
    np.concatenate(gradient_divergence,axis=1)
    
    
    return gradient_divergence

#prev testing block with user input 
#gradient tol:
tol_len=0.005 #meter resolution for length measurements using metal calipers
err_spec=1/100 #1 percent error spec
tol_grid=2.54/500 #inches to meters / 5 squares 1" for grid res
fret_info=[10,10,5] #neck stock len, scale len, max frets over est
fb_width=3.5*2.54/100#inches to m
fret_locs,fret_nums,true_max_frets=scale_len_fret_locs(fret_info)
mesh_refinement_params=[tol_len,tol_grid,err_spec,true_max_frets]
max_grid_estimate=round(fb_width*0.25/tol_grid)
all_fret_params=chug_polygon_props(true_max_frets,max_grid_estimate)
#doesn't liek how i try to index numpy array from list of lists
all_fret_params=(all_fret_params)
#expects 4 values to unpack
#[A, Cx, Cy, Ix, Iy, Ixy] output of polygon props, 

#list indices must be integers or slices, not tuple
    #need to index all fret params in each row, then index the columns i want from that
        #this is considered tuple [:,0 ] tf would a slice be
        #
[area_eff,moi_transv_eff,moi_double_norm_eff,moi_longitudinal_eff]=[all_fret_params[0:5:-1],all_fret_params[3:5:-1],all_fret_params[4:5:-1],all_fret_params[5:5:-1]]
mois=[moi_transv_eff,moi_double_norm_eff,moi_longitudinal_eff]

'''
#dummy props data 
area_eff=np.random.rand(true_max_frets)*10+50
moi_eff=np.random.rand(true_max_frets)*1000+5000
props=[fret_nums,fret_locs,area_eff,moi_eff]
'''
props=[fret_nums,fret_locs,area_eff,mois]
mesh_advice(mesh_refinement_params,props=props)

In [ ]:
#tests 
#cw order
#wikipedia refs
#square 1 len
vertex=[[0,0],[0,1],[1,1],[1,0]]
print(f"Known Params:\n A: {1}\n Ix: {1/12}\n Iy: {1/12}\n C: {0.5,0.5}\n")
[A, Cx, Cy, Ix, Iy, Ixy]=polygon_centroid_properties(vertex)
print(A,Ix,Iy,Cx,Cy)
#rect 13h 5 b
vertex=[[0,0],[5,0],[5,13],[0,13]]
print(f"Known Params: A:\n {13*5}\n Ix: {5*(13**3)/12}\n Iy: {(5**3)*13/12}\n C: {2.5,6.5}\n")
[A, Cx, Cy, Ix, Iy, Ixy]=polygon_centroid_properties(vertex)
print(A,Ix,Iy,Cx,Cy)
#3 4 5 triangle
vertex=[[0,0],[4,0],[0,3]]
print(f"Known Params: A:\n {6}\n Ix: {4*(3**3)/36}\n Iy: {3*(4**3)/36}\n C: {4/3,1}\n")
[A, Cx, Cy, Ix, Iy, Ixy]=polygon_centroid_properties(vertex)
print(A,Ix,Iy,Cx,Cy)
#passes